We have a dataset (DCC) with a shortage of certain labels. We want to generate 
new samples synthetically using GPT-4. We will use the following approach:
1. We take the existing samples for each document type and present these to GPT-4
2. we ask to generate new sentences like it, where the token labels are provided in the BIO format

We care specifically about the following labels:
* Experiencer: Other
* Historical: Hypothetical

The task of the GPT model is to generate new sentences that are similar to the input sentences but with variations of the medical concepts. 




# Definitions
The definitions are taken from the ConText/ConTextD papers:

## Negation

This property has two values, ‘Negated’ or ‘Not negated’. A clinical condition or term is labeled as ‘Negated’ if there is evidence in the text suggesting that the condition does not occur or exist, e.g., ‘There was no sign of sinus infection’, otherwise it is ‘Not negated’.

## Temporality

The temporality property places a condition along a time line. There are three possible values for this property: ‘Recent’, ‘Historical’, and ‘Hypothetical’. A condition is considered ‘Recent’ if it is maximally 2 weeks old. Conditions that developed more than 2 weeks ago are labeled as ‘Historical’. A condition is labeled as ‘Hypothetical’ if it is not ‘Recent’ or ‘Historical’, e.g., ‘patient should return if she develops fever’ [13].

**Adaptation**: *'Hypothetical' is specifically about theoretical concepts, that are not (yet) realized. 'Historical' and 'Recent' 
can be used for realized concepts, in which we also include their negations. I.e. if a concept is explicitly denied historically or recently, we can label it as 'Historical' or 'Recent' respectively.*

## Experiencer

Clinical text may refer to subjects other than the actual patient. The experiencer property describes whether the patient experienced the condition or someone else. For simplicity, we have defined only two possible values for this property: ‘Patient’ or ‘Other’, where ‘Other’ refers to anyone but the actual patient, e.g., ‘Mother is recently diagnosed with cancer’.

In [448]:
import os, sys, re
import json, dotenv
import pprint

import openai
import asyncio
from openai import AsyncOpenAI, OpenAI
from tqdm import tqdm
from collections import defaultdict

import pandas as pd

dotenv.load_dotenv()

True

In [3]:
openai.api_key = os.getenv("OPENAI_KEY")

In [397]:
DCC = json.load(open('../data/emc-dcc_ann_NEW.json'))

In [398]:
docs = DCC['projects'][0]['documents']

In [399]:
class_counts = {'Negation': defaultdict(int),
                'Temporality': defaultdict(int),
                'Experiencer': defaultdict(int)}

for doc in docs:
    for ann in doc['annotations']:
        for _class, val in ann['meta_anns'].items():
            class_counts[_class][val['value']] += 1

In [400]:
pprint.pprint(class_counts, indent=2)

{ 'Experiencer': defaultdict(<class 'int'>, {'patient': 12455, 'other': 96}),
  'Negation': defaultdict(<class 'int'>,
                          { 'negated': 1760,
                            'not negated': 10791}),
  'Temporality': defaultdict(<class 'int'>,
                             { 'historical': 513,
                               'hypothetical': 106,
                               'recent': 11932})}


In [401]:
relevant_docs_hypothetical = []
for i, doc in enumerate(docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Temporality']['value']=='hypothetical'):
            doc['index'] = i
            relevant_docs_hypothetical.append(doc)
            break
        
relevant_docs_experiencer = []
for i, doc in enumerate(docs):
    for concept in doc['annotations']:
        if (concept['meta_anns']['Experiencer']['value']=='other'):
            doc['index'] = i
            relevant_docs_experiencer.append(doc)
            break


In [309]:
len(relevant_docs_hypothetical), len(relevant_docs_experiencer)

(78, 79)

In [402]:
relevant_docs_hypothetical[69]['text'][93:129]

'koorts niet gemeten.\n'

In [356]:
[(d['start'],d['end'], d['id']) for d in relevant_docs_hypothetical[69]['annotations'] 
 if d['meta_anns']['Temporality']['value']=='hypothetical']

[(93, 99, 16597)]

In [277]:
relevant_docs_hypothetical[9]['name']

'DL1814'

In [284]:
corrections = [
    {'doc_id': 'DL1616', 'annotation_id': 1873, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'DL1139', 'annotation_id': 108, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'GP2799', 'annotation_id': 8210, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'SP1476', 'annotation_id': 15532, 'meta': 'Experiencer', 'value': 'patient'},
    {'doc_id': 'DL1567', 'annotation_id': 1694, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1711', 'annotation_id': 2232, 'meta': 'Temporality', 'value': 'recent'},
    {'doc_id': 'DL1812', 'annotation_id': 2232, 'meta': 'Temporality', 'value': 'historical'},
    {'doc_id': 'DL1814', 'annotation_id': 2703, 'meta': 'Temporality', 'value': 'historical'},
 ]

In [290]:
# update the docs
update_docs = docs.copy()
for c in corrections:
    for d in update_docs:
        if d['name']==c['doc_id']:
            for a in d['annotations']:
                if a['id']==c['annotation_id']:
                    a['meta_anns'][c['meta']]['value'] = c['value']
# put updated docs in DCC
DCC['projects'][0]['documents'] = update_docs

In [294]:
# write DCC back to json 
with open('../data/emc-dcc_ann_NEW.json', 'w') as f:
    json.dump(DCC, f, indent=2)

In [285]:
OAI_ASYNC_CLIENT = AsyncOpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)
OAI_CLIENT = OpenAI(api_key=os.getenv("OPENAI_KEY"), max_retries=2)

In [438]:
SYSTEM_PROMPT_HYPOTHETICAL = """
    Je bent een kritische assistent die mij helpt om nieuwe zinnen te bedenken.
    Deze zinnen moeten voldoen aan de volgende eisen:
    - ze moeten semantisch correct zijn en vergelijkbaar zijn met de voorbeeldzinnen die ik je geef.
    - de voorbeeldzinnen worden voorafgegaan door de term VOORBEELDZIN
    - in de voorbeeldzin worden 1 of meer concepten benoemd die hypothethisch zijn, het is belangrijk
    dat deze concepten in de nieuwe zin ook hypothetisch zijn, het mogen ook andere concepten zijn. 
    Een voorbeeld van een hypothetische concept = 'een voorafgaand trauma kan niet worden herinnerd', waarin 'trauma' het concept is.
    Een ander voorbeeld = 'ter uitsluiting van epifysaire dysplasie' waarin 'epifysaire dysplasie' het concept is.
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - geef als antwoord ALLEEN de nieuw gegenereerde zinnen, voorafgaand met de term NIEUWE_ZIN
    - in de NIEUWE_ZIN, plaats de concepten die hypothetisch zijn tussen verticale streepjes, dus '|', 
    dus bijvoorbeeld: 'ter uitsluiting van |epifysaire dysplasie|'
"
"""

SYSTEM_PROMPT_EXPERIENCER = """
    Je bent een kritische assistent die mij helpt om nieuwe text te bedenken.
    Deze text moeten voldoen aan de volgende eisen:
    - het moet semantisch correct zijn en vergelijkbaar zijn met de text die ik je geef.
    - de voorbeeldtext wordt voorafgegaan door de term VOORBEELDTEXT
    - in de voorbeeldtext worden 1 of meer concepten benoemd die verwijzen naar een persoon anders dan de patient, het is belangrijk
    dat deze concepten in de nieuwe zin ook verwijzen naar iemand anders dan de patient (zoals een familielid), 
    het mogen ook andere medische concepten zijn.
    - de concepten die je moet vervangen zijn aangegeven met vertical streepjes, dus |concept|.
    - Een voorbeeld van een concept wat verwijst naar een ander persoon dan de patient =
    'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella luxaties', waarin 'luxaties' het concept is, en er 
    wordt verwezen naar de zus van de patient.    
    - het domein is medisch dus gebruik medische concepten.
    - probeer de medische concepten te varieren, dus gebruik niet steeds dezelfde concepten.
    - varieer de ziektebeelden
    - varieer de opmaak van de text, dus gebruik niet steeds dezelfde opmaak.
    - geef als antwoord ALLEEN de nieuw gegenereerde text, voorafgaand met de term NIEUWE_TEXT
    - in de NIEUWE_TEXT, plaats de concepten die verwijzen naar een ander persoon dan de patient tussen tussen verticale streepjes |, 
    dus bijvoorbeeld: 'Een zusje van #Name# is elders operatief behandeld in verband met recidiverende patella |luxaties|'
"""

In [299]:
def get_chat_res(USER_TEXT='Good day', 
                 SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                 n = 10,
                 MODEL="gpt-4"):
    return OAI_CLIENT.chat.completions.create(
            model=MODEL,
            n = n,
            messages=[
                        {"role": "system",
                        "content": SYSTEM_PROMPT
                        },
                        {"role": "user", 
                        "content": USER_TEXT
                        }],
            stream=False,
        )

In [300]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
nieuwe_zinnen_hypothetisch = []
for i, doc in tqdm(enumerate(relevant_docs_hypothetical)):
    res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_HYPOTHETICAL, 
                       n=1,
                       USER_TEXT="Parafraseer de VOORBEELDZIN: " + doc['text'])
    # extract the part after "NIEUWE_ZIN"
    for _res in res.choices:
        txt = _res.message.content
        nieuwe_zinnen_hypothetisch.append(txt[txt.find('NIEUWE_ZIN:')+12:].strip())
        
    break

0it [00:04, ?it/s]


In [451]:
#re_extract = re.compile(r'NIEUWE_ZIN\:(.*)')
nieuwe_zinnen_experiencer = []
for i, doc in tqdm(enumerate(relevant_docs_experiencer)):
    EXAMPLE = doc['text'].replace('|', ' ')
    # add | vertical bars around the concept that needs to be replaced
    LOCS = [(d['start'],d['end']) for d in doc['annotations'] 
                if d['meta_anns']['Experiencer']['value']=='other']
    for loc in LOCS:
        EXAMPLE = EXAMPLE[:loc[0]] + '|' + EXAMPLE[loc[0]:loc[1]] + '|' + EXAMPLE[loc[1]:]
    
    res = get_chat_res(SYSTEM_PROMPT=SYSTEM_PROMPT_EXPERIENCER, 
                       n=10,
                       USER_TEXT="VOORBEELDTEXT: " + EXAMPLE)

    for j, _res in enumerate(res.choices):
        txt = _res.message.content
        nieuwe_zinnen_experiencer.append((
            doc['name'],
            j,
            txt[txt.find('NIEUWE_TEXT')+12:].strip()))

0it [00:00, ?it/s]

In [ ]:
res = pd.DataFrame(nieuwe_zinnen_experiencer, columns=['doc_id', 'synth_num', 'text'])
res.to_parquet('../data/synth_experiencer.parquet')

,doc_id,synth_num,text
0,133,0,De broer van #Name# is in een ander ziekenhuis...
1,335,0,In de familiegeschiedenis is geen sprake van |...
2,338,0,Geschiedenis: Tijdens een routine controle bij...
3,363,0,Het is mogelijk dat de biologische grootvader ...


### Extract the spans from the synthetic set and add them to the original dataset with an additional label

## Write new samples to the dataset

## Translate English corpora

* English -- BioScope; [HF](https://huggingface.co/datasets/bigbio/bioscope), [src](https://rgai.inf.u-szeged.hu/downloads)
* English -- [Genia](http://www.geniaproject.org/genia-corpus/term-corpus)
* English -- Sherlock, SFU review corpus
